In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
train=pd.read_csv("/Users/hemanth/Downloads/ipl/deliveries.csv")

In [3]:
pd.set_option("max_columns",150)
pd.set_option("max_rows",150)

ind=train[train["is_super_over"]==1].index
train.drop(ind,axis=0,inplace=True)

In [4]:
players=train["bowler"].unique()
players2=train["batsman"].unique()

In [5]:
test=pd.DataFrame()
test["players"]=players
test=pd.concat([test,pd.DataFrame(data=players2,columns=["players"])])

In [6]:
test.drop_duplicates(inplace=True)
test.reset_index(drop=True,inplace=True)

In [7]:
check=train[["batsman","match_id"]]
check.drop_duplicates(inplace=True)
li=check["batsman"].value_counts()
test2=pd.DataFrame(data=li)
test2=test2.reset_index()
test2.columns=[["players","bat_inn"]]
test=pd.merge(test,test2,on=["players"],how="left")

/Users/hemanth/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
check=train[["bowler","match_id"]]
check.drop_duplicates(inplace=True)
li=check["bowler"].value_counts()
test2=pd.DataFrame(data=li)
test2=test2.reset_index()
test2.columns=[["players","bowl_inn"]]
test=pd.merge(test,test2,on=["players"],how="left")

/Users/hemanth/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
li=[]
import random
for i in range(0,495):
    li.append(random.randint(3,12))
test["matches"]=pd.Series(li)
for i in range(0,495):
    test["matches"].iloc[i]=test["matches"].iloc[i]+max(test["bat_inn"].iloc[i],test["bowl_inn"].iloc[i])

/Users/hemanth/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [10]:
check=train.groupby(["bowler"],as_index=False)["total_runs"].sum()
check.columns=["players","total_runs_bowl"]
test=pd.merge(test,check,on=["players"],how="left")

In [11]:
check=train.groupby(["batsman"],as_index=False)["batsman_runs"].sum()
check.columns=["players","total_runs_bat"]
test=pd.merge(test,check,on=["players"],how="left")

In [12]:
check=train.groupby(["bowler"],as_index=False).count()[["bowler","ball"]]
check["ball"]=check["ball"]/6
check.columns=["players","over"]
test=pd.merge(test,check,on=["players"],how="left")

In [13]:
check=train[~train["player_dismissed"].isnull()]
check=check.groupby(["bowler"],as_index=False).count()[["bowler","match_id"]]
check.columns=["players","wickets"]
test=pd.merge(test,check,on=["players"],how="left")

In [14]:
check=train[~train["player_dismissed"].isnull()]
check=check.groupby(["batsman"],as_index=False).count()[["batsman","match_id"]]
check.columns=["players","outs"]
test=pd.merge(test,check,on=["players"],how="left")

In [15]:
test["bat_avg"]=test["total_runs_bat"]/test["outs"]

In [16]:
check=train[~train["player_dismissed"].isnull()]
check=check.groupby(["bowler","match_id"],as_index=False).count()[["bowler","inning"]]
check=check[check["inning"]>3]
check=check.groupby(["bowler"],as_index=False).count()
check.columns=["players","three_wicket"]
test=pd.merge(test,check,on=["players"],how="left")

In [17]:
check=train.groupby(["batsman","match_id"],as_index=False)["total_runs"].count()
check=check[check["total_runs"]>30]
check=check.groupby(["batsman"],as_index=False)["total_runs"].count()
check.columns=["players","no_30"]
test=pd.merge(test,check,on=["players"],how="left")

In [18]:
check=train.groupby(["batsman","match_id"],as_index=False)["total_runs"].count()
check=check[check["total_runs"]>60]
check=check.groupby(["batsman"],as_index=False)["total_runs"].count()
check.columns=["players","no_60"]
test=pd.merge(test,check,on=["players"],how="left")

In [19]:
test=test.fillna(0)

In [20]:
test["bowl_u"]=np.sqrt(test["bowl_inn"]/test["matches"])
test["bowl_v"]=5*test["three_wicket"]+test["wickets"]
test["bowl_avg"]=test["total_runs_bowl"]/test["wickets"]
test["bowl_econ"]=test["total_runs_bowl"]/test["over"]
test["bowl_w"]=test["bowl_avg"]*test["bowl_econ"]

In [21]:
check=train.groupby(["batsman","match_id"],as_index=False).sum()[["batsman","match_id","batsman_runs"]]
check=check.groupby(["batsman"]).head(4)
check=check.groupby(["batsman"],as_index=False).mean()[["batsman","batsman_runs"]]
check.columns=["players","rec_score"]
test=pd.merge(test,check,on=["players"],how="left")

In [22]:
test["bat_u"]=np.sqrt(test["bat_inn"]/test["matches"])
test["bat_v"]=test["no_60"]*10+test["no_30"]*5
test["bat_w"]=test["bat_v"]*0.3+test["bat_avg"]*0.7

In [23]:
test["bowl_score"]=(test["bowl_u"]*test["bowl_v"])/test["bowl_w"]
test["carrer_score"]=test["bat_u"]*test["bat_w"]

In [24]:
test=test.replace(np.inf, 0)
test=test.fillna(0)

In [25]:
max_car=np.max(test["carrer_score"])
max_rec=np.max(test["rec_score"])

test["carrer_score"]=test["carrer_score"]/max_car
test["rec_score"]=test["rec_score"]/max_rec

test["bat_score"]=test["carrer_score"]*0.4+test["rec_score"]*0.6

In [26]:
test

,players,bat_inn,bowl_inn,matches,total_runs_bowl,total_runs_bat,over,wickets,outs,bat_avg,three_wicket,no_30,no_60,bowl_u,bowl_v,bowl_avg,bowl_econ,bowl_w,rec_score,bat_u,bat_v,bat_w,bowl_score,carrer_score,bat_score
0,TS Mills,3.0,5.0,8.0,157.0,8.0,18.500000,5.0,3.0,2.666667,0.0,0.0,0.0,0.790569,5.0,31.400000,8.486486,266.475676,0.041184,0.612372,0.0,1.866667,0.014834,0.010904,0.029072
1,A Choudhary,3.0,5.0,17.0,144.0,25.0,18.000000,5.0,2.0,12.500000,0.0,0.0,0.0,0.542326,5.0,28.800000,8.000000,230.400000,0.128700,0.420084,0.0,8.750000,0.011769,0.035061,0.091245
2,YS Chahal,8.0,55.0,64.0,1600.0,15.0,203.166667,72.0,2.0,7.500000,1.0,0.0,0.0,0.927025,77.0,22.222222,7.875308,175.006836,0.019305,0.353553,0.0,5.250000,0.407875,0.017705,0.018665
3,S Aravind,10.0,38.0,43.0,1057.0,59.0,131.333333,48.0,3.0,19.666667,2.0,0.0,0.0,0.940064,58.0,22.020833,8.048223,177.228585,0.077220,0.482243,0.0,13.766667,0.307646,0.063326,0.071662
4,SR Watson,98.0,94.0,101.0,2490.0,2622.0,326.833333,100.0,84.0,31.214286,2.0,19.0,2.0,0.964724,110.0,24.900000,7.618562,189.702193,0.235521,0.985037,115.0,56.350000,0.559401,0.529458,0.353096
5,TM Head,10.0,5.0,19.0,81.0,205.0,8.000000,3.0,6.0,34.166667,0.0,1.0,0.0,0.512989,3.0,27.000000,10.125000,273.375000,0.239382,0.725476,5.0,25.416667,0.005630,0.175884,0.213983
6,STR Binny,56.0,57.0,67.0,686.0,766.0,92.333333,27.0,37.0,20.702703,0.0,1.0,0.0,0.922359,27.0,25.407407,7.429603,188.766947,0.196911,0.914232,5.0,15.991892,0.131928,0.139457,0.173930
7,A Nehra,17.0,88.0,97.0,2537.0,41.0,329.000000,121.0,8.0,5.125000,3.0,0.0,0.0,0.952479,136.0,20.966942,7.711246,161.681253,0.015444,0.418638,0.0,3.587500,0.801189,0.014326,0.014997
8,B Kumar,33.0,90.0,95.0,2410.0,158.0,342.333333,117.0,18.0,8.777778,3.0,0.0,0.0,0.973329,132.0,20.598291,7.039922,145.010361,0.027027,0.589380,0.0,6.144444,0.886001,0.034543,0.030034
9,BCJ Cutting,8.0,9.0,13.0,234.0,124.0,28.500000,9.0,5.0,24.800000,0.0,0.0,0.0,0.832050,9.0,26.000000,8.210526,213.473684,0.227799,0.784465,0.0,17.360000,0.035079,0.129900,0.188639
